# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [14]:
import numpy as np
import os
import cv2
import datetime
import os
import matplotlib.pyplot as plt

We set the random seed so that the results don't vary drastically.

In [15]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
#tf.random.set_seed(30)
tf.set_random_seed(30) # GPU

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [16]:
train_doc = (np.random.permutation(open(r'./Project_data/train.csv').readlines()))
val_doc = np.random.permutation(open(r'./Project_data/val.csv').readlines())

batch_size = 30 #experiment with the batch size

In [17]:
print(train_doc.shape)
print(val_doc.shape)
train_shape = train_doc.shape[0]

(663,)
(100,)


In [18]:
train_doc.shape[0]

663

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [19]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    image_seq=18 # no. of images seq in a video
    img_idx = [2,4,6,8,10,12,13,14,15,16,17,18,20,21,22,24,26,28] #create a list of image numbers you want to use for a particular video
    image_height=90 # height
    image_width=90 # width 

    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)//batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,image_seq,image_height,image_width,3)) 
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes                 
                    
                    if(image.shape[1]==160):
                        image_resized = cv2.resize(image[:,20:140,:],(image_width,image_height))
                    else:
                        image_resized = cv2.resize(image,(image_width,image_height))


                    batch_data[folder,idx,:,:,0] = (image_resized[:,:,0])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image_resized[:,:,1])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image_resized[:,:,2])/255 #normalise and feed in the image

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        # write the code for the remaining data points which are left after full batches

        pending_folders = len(t)%batch_size
        if ( pending_folders > 0):

            batch_data = np.zeros((pending_folders,image_seq,image_height,image_width,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((pending_folders,5)) # batch_labels is the one hot representation of the output

            for folder in range(pending_folders): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = cv2.imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    if(image.shape[1]==160):
                        image_resized = cv2.resize(image[:,20:140,:],(image_width,image_height))
                    else:
                        image_resized = cv2.resize(image,(image_width,image_height))

                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes

                    batch_data[folder,idx,:,:,0] = (image_resized[:,:,0])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image_resized[:,:,1])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image_resized[:,:,2])/255 #normalise and feed in the image

                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [20]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'

num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [21]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout ,GlobalAveragePooling3D
from keras.layers.convolutional import Conv3D, MaxPooling3D 

from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam, SGD, RMSprop
from keras.regularizers import l2

#write your model here
input_shape =[18,90,90,3]

model = Sequential()
# first conv layer
model.add(Conv3D(32, kernel_size=(3, 3, 3),
                 activation='relu',
                 input_shape=input_shape,kernel_regularizer=l2(0.01), padding='same')) # input shape = (img_rows, img_cols, 1)
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2)))

# second conv layer
model.add(Conv3D(64, kernel_size=(3, 3, 3), 
                 activation='relu',kernel_regularizer=l2(0.01), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Dropout(0.25))

# third conv layer
model.add(Conv3D(128, kernel_size=(3, 3, 3), 
                 activation='relu',kernel_regularizer=l2(0.01), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Dropout(0.25))

# fourth conv layer
model.add(Conv3D(256, kernel_size=(3, 3, 3), 
                 activation='relu',kernel_regularizer=l2(0.01), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2,2,2)))
model.add(Dropout(0.25))


# flatten and put a fully connected layer
model.add(Flatten())
model.add(Dense(256, activation='relu',kernel_regularizer=l2(0.01)))
# fully connected with regularization
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [22]:
#optimiser = SGD(learning_rate=0.001, momentum=0.5, nesterov=True)#write your optimizer #lr for GPU
optimiser = SGD(lr=0.001, momentum=0.5, nesterov=True)
#optimiser = Adam(lr=0.001)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 18, 90, 90, 32)    2624      
_________________________________________________________________
batch_normalization_5 (Batch (None, 18, 90, 90, 32)    128       
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 9, 45, 45, 32)     0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 9, 45, 45, 64)     55360     
_________________________________________________________________
batch_normalization_6 (Batch (None, 9, 45, 45, 64)     256       
_________________________________________________________________
max_pooling3d_6 (MaxPooling3 (None, 4, 22, 22, 64)     0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 4, 22, 22, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [23]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [24]:

model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early_Stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=3)# write the REducelronplateau code here

callbacks_list = [checkpoint, LR,early_Stopping]
#callbacks_list = [LR,early_Stopping]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [25]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [26]:

start_dt_time = datetime.datetime.now()

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

end_dt_time = datetime.datetime.now()
print("Total time taken --> ",(end_dt_time-start_dt_time))

Source path =  ./Project_data/val ; batch size = 30
Source path =  ./Project_data/train ; batch size = 30
Epoch 1/30
23/23 [==============================] - 27s 1s/step - loss: 11.1437 - categorical_accuracy: 0.2773 - val_loss: 9.5591 - val_categorical_accuracy: 0.4800

Epoch 00001: val_loss improved from inf to 9.55906, saving model to model_init_2020-12-2313_56_53.773365/model-00001-11.18890-0.28808-9.55906-0.48000.h5
Epoch 2/30
23/23 [==============================] - 21s 919ms/step - loss: 9.8741 - categorical_accuracy: 0.3745 - val_loss: 9.4308 - val_categorical_accuracy: 0.4000

Epoch 00002: val_loss improved from 9.55906 to 9.43079, saving model to model_init_2020-12-2313_56_53.773365/model-00002-9.85899-0.38914-9.43079-0.40000.h5
Epoch 3/30
23/23 [==============================] - 23s 990ms/step - loss: 9.3983 - categorical_accuracy: 0.4315 - val_loss: 9.1669 - val_categorical_accuracy: 0.5400

Epoch 00003: val_loss improved from 9.43079 to 9.16689, saving model to model_init_